In [1]:
from datetime import timedelta
import datetime
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
pd.set_option('display.width', 1000)
#from sklearn.model_selection import train_test_split
#from sklearn import metrics
#from imblearn.over_sampling import SMOTE
#from sklearn.metrics import precision_score, recall_score, confusion_matrix, accuracy_score, roc_auc_score, f1_score, roc_curve, auc,precision_recall_curve
#from sklearn.linear_model import LogisticRegression
#import pickle


In [2]:
loan = pd.read_csv('loan.asc','delimiter',';')

In [3]:
loan['datetime'] = pd.to_datetime(loan['date'], unit='s')

In [4]:
loan[['datetime']].describe()

<ipython-input-4-8dbbb8522cd4>:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  loan[['datetime']].describe()


,datetime
count,682
unique,559
top,1970-01-12 05:47:08
freq,4
first,1970-01-11 18:31:45
last,1970-01-12 08:33:28


In [7]:
def _translate_and_clean(account,card,client,disp,district,loan,order,trans):
    # Basic preprocessing / initial feature engineering:
    # Translate from Czech to English, undo funny formats like for gender, add a few agg stats...

    def num2date(x):
        if isinstance(x, str):
            return pd.to_datetime('19'+x, format='%Y%m%d %H:%M:%S')
        else:
            return pd.to_datetime(str(float(x)+19000000.), format='%Y%m%d')


    # Account:
    account['date'] = account['date'].apply(lambda x: num2date(x))
    account['frequency'].replace('POPLATEK MESICNE','monthly',inplace=True)
    account['frequency'].replace('POPLATEK TYDNE','weekly',inplace=True)
    account['frequency'].replace('POPLATEK PO OBRATU','after_tr',inplace=True) # after transaction
    account.rename(columns = {'frequency':'stmt_frq'}, inplace=True) # statement freq
    # Card:
    card['issued'] = card['issued'].apply(lambda x: num2date(x))
    card.rename(columns = {'issued':'date'}, inplace=True) # date credit card issued
    # Client:
    client['MM']=client['birth_number']//100 - client['birth_number']//10000*100
    client['gender'] = 'M'
    client.loc[client['MM']>50,'gender'] = 'F'
    client.loc[client['gender']=='F','birth_number'] -= 5000
    client['birth_number'] = client['birth_number'].apply(lambda x: num2date(x))
    client.rename(columns = {'birth_number':'date_birth'}, inplace=True) # client's birthdate
    client.drop('MM',1,inplace=True)
    # Disp:
    disp['type'].replace('OWNER','owner',inplace=True)
    disp['type'].replace('DISPONENT','disponent',inplace=True)
    # District:
    district.rename(columns = {
    'A1':'district_id','A2':'dname','A3':'region','A4':'pop','A5':'nmu500','A6':'nmu2k',
    'A7':'nmu10k','A8':'nmuinf','A9':'ncit','A10':'rurba','A11':'avgsal',
    'A12':'urat95','A13':'urat96','A14':'ent_ppt','A15':'ncri95','A16':'ncri96'}, inplace=True)
    # Loan:
    loan['date'] = loan['date'].apply(lambda x: num2date(x))
    # Order:
    order['k_symbol'].replace('POJISTNE','ins_paymt',inplace=True) # insurrance payment
    order['k_symbol'].replace('SIPO','household',inplace=True)
    order['k_symbol'].replace('LEASING','leasing',inplace=True)
    order['k_symbol'].replace('UVER','loan_payt',inplace=True) # loan payment
    order.rename(columns = {'k_symbol':'category'}, inplace=True)
    # Trans  # takes ~5min on my mbp w/ 16gb ram
    trans['date'] = trans['date'].apply(lambda x: num2date(x))
    trans['type'].replace('PRIJEM','credit',inplace=True)
    trans['type'].replace('VYDAJ','withdrawal',inplace=True)
    trans['operation'].replace('VYBER KARTOU','creditcard_wd',inplace=True) # credit card withdrawal
    trans['operation'].replace('VKLAD','credit_in_cash',inplace=True)
    trans['operation'].replace('PREVOD Z UCTU','coll_from_bank',inplace=True) # collection from another bank
    trans['operation'].replace('VYBER','cash_wd',inplace=True) # cash withdrawal
    trans['operation'].replace('PREVOD NA UCET','remi_to_bank',inplace=True) # remittance to another bank
    trans['k_symbol'].replace('POJISTNE','ins_paymt',inplace=True) # insurrance payment
    trans['k_symbol'].replace('SLUZBY','paymt_for_stmt',inplace=True) # payment for statement(?)
    trans['k_symbol'].replace('UROK','int_credited',inplace=True) # interest credited
    trans['k_symbol'].replace('SANKC. UROK','sanc_int',inplace=True) # sanction interest for neg balance
    trans['k_symbol'].replace('SIPO','household',inplace=True)
    trans['k_symbol'].replace('DUCHOD','pension',inplace=True) # old-age pension
    trans['k_symbol'].replace('UVER','loan_paymt',inplace=True) # loan payment
    trans.rename(columns = {'k_symbol':'category'}, inplace=True)
    # Note the snafu that pandas int columns can't contain NaNs, but the
    # trans.account (destination account#) field does, so it's type float.
    # But account_id column is always filled so it's an integer.

    return account,card,client,disp,district,loan,order,trans

In [8]:
def get_bank_data():
    '''
    Read, process, and provide the PKDD99 bank transactions data.
    Parameters
    ----------
    (none)
    Returns
    -------
    account, card, client, disp, district, loan, order, trans : Pandas dataframe
        The translated contents of the original PKDD99 dataset, contained in
        Panads dataframes.  Further details are available in meta-data attached
        to these dataframes in the .notes and .description attributes, e.g.
        account.notes.  Additionally the loan dataframe has a meta-data
        attribute of codes, i.e. loan.codes, returning the status definitions.
        The rest of the details are in the dataset's original data description
        document in the references.
    Notes
    -----
    Assumes the files account.asc, card.asc, client.asc, disp.asc, district.asc,
    loan.asc, order.asc, trans.asc from the original data distribution exist in
    the current working directory.
    Examples
    --------
    >>> import pkdd99_bank_data as pkdd99
    >>> account,card,client,disp,district,loan,order,trans = pkdd99.get_bank_data()
    References
    ----------
    Original data description document with further info about fields/format:
    http://sorry.vse.cz/~berka/challenge/pkdd1999/berka.htm
    '''
    # FIXME: ideally should have a check first that these files exist...
    account = pd.read_csv('account.asc','delimiter',';')
    card = pd.read_csv('card.asc','delimiter',';')
    client = pd.read_csv('client.asc','delimiter',';')
    disp = pd.read_csv('disp.asc','delimiter',';')
    district = pd.read_csv('district.asc','delimiter',';')
    loan = pd.read_csv('loan.asc','delimiter',';')
    order = pd.read_csv('order.asc','delimiter',';')
    trans = pd.read_csv('trans.asc','delimiter',';',low_memory=False)

    account.name = 'Account'
    card.name = 'Card'
    client.name = 'Client'
    disp.name = 'Disp'
    district.name = 'District'
    loan.name = 'Loan'
    order.name = 'Order'
    trans.name = 'Trans'

    # descriptions are cut/pasted from the Financial Data Description webpage for the data
    account.description = 'each record describes static characteristics of an account'
    card.description = 'each record describes a credit card issued to an account'
    client.description = 'each record describes characteristics of a client'
    disp.description = 'each record relates together a client with an account'
    district.description = 'each record describes demographic characteristics of a district'
    loan.description = 'each record describes a loan granted for a given account'
    order.description = 'each record describes characteristics of a payment order'
    trans.description = 'each record describes one transaction on an account'

    account.notes = '(one account can have one or more clients, e.g. married couples)'
    card.notes = '(one account can have one or more credit cards)'
    client.notes = '(one client can have one or more accounts)'
    disp.notes = '(disposition connects a client/account pair and allows to link one or more cards)'
    district.notes = '(neighborhoods for both bank/account branches and client homes.  same 16 fields as original A1-16.)'
    loan.notes = '(one account may have zero or one loan.  see loan.codes for ABCD status definitions)'
    order.notes = '(one payment order is from one account)'
    trans.notes = '(category,bank,account are NaN for some types/operations.)'
    loan.codes = 'Loan status codes:\nA = contract finished, no problems\nB = contract finished, loan not payed\nC = running contract, OK so far\nD = running contract, client in debt'

    account,card,client,disp,district,loan,order,trans = _translate_and_clean(
        account,card,client,disp,district,loan,order,trans
    )

    return account,card,client,disp,district,loan,order,trans

In [9]:
account,card,client,disp,district,loan,order, trans = get_bank_data()

In [10]:
loan.head()

,loan_id,account_id,date,amount,duration,payments,status
0,5314,1787,1993-07-05,96396,12,8033.0,B
1,5316,1801,1993-07-11,165960,36,4610.0,A
2,6863,9188,1993-07-28,127080,60,2118.0,A
3,5325,1843,1993-08-03,105804,36,2939.0,A
4,7240,11013,1993-09-06,274740,60,4579.0,A


In [16]:
account.head()

,account_id,district_id,stmt_frq,date
0,576,55,monthly,1993-01-01
1,3818,74,monthly,1993-01-01
2,704,55,monthly,1993-01-01
3,2378,16,monthly,1993-01-01
4,2632,24,monthly,1993-01-02


In [19]:
loan.to_csv('.//dados_tratados_usaveis//loan.csv', index=False, decimal=".", sep=";")

In [18]:
account.to_csv('.//dados_tratados_usaveis//account.csv', index=False, decimal=".", sep=";")

In [ ]:

def read_df_asc(tabela=str):
    """
    Função para as bases de dados onde retorna no print o 'shape', um breve 'show' e o Scheema das variáveis.
    :param entidade_name: string que referencie o nome da tabela que complete o caminho './original_data/{tabela}.asc'. 
    tabela pode ser => account, card, client, disp, district, loan, order 
    :return: DataFrame em pyspark
    """
    df = spark.read.csv(path = f'./original_data/{tabela}.asc', header='True',inferSchema='False', sep=';')
    print('\n','A base de dados possui:',df.count(), 'linhas', 'e', len(df.columns), 'colunas', '\n')
    print(df.show(5))
    print(df.printSchema())
    return(df)

In [ ]:
loan_df = read_df_asc('loan')

In [ ]:
spark.conf.set("spark.sql.session.timeZone", "Europe/Prague")

In [ ]:
loan_df1 = (
    loan_df
    .withColumn("date", f.from_unixtime("date"))
    .withColumnRenamed("date", "date_loan")
    )
loan_df1.show(truncate=False)

In [ ]:
account_df = read_df_asc('account')

In [ ]:
account_df.show()

In [ ]:
account_df1 =(
    account_df
    .withColumn("date", f.from_unixtime("date"))
    .withColumnRenamed("date", "date_account")
)
account_df1.show(truncate=False)

## Loan and Account

* 606 good loans and 76 bad
* Features extracted:
   - loan amount
   - loan duration
   - loan payment
   - account district id
   - frequency
   - date loan issued
   - date account opened
   - days between

In [ ]:
print(loan_df1.count())
print(account_df1.count())

In [ ]:
df = account_df1.join(loan_df1, on="account_id", how="inner")
df = df.withColumn("days_between", f.datediff("date_loan", "date_account"))


In [ ]:
df.select('days_between').summary().show()

In [ ]:
df.show()

## Definição de bom e mau

In [ ]:
df_good = df.filter((f.col("status") == "A") | (f.col("status") == "C"))
df_bad = df.filter((f.col("status") == "B") | (f.col("status") == "D"))

In [ ]:
print(df_good.count())
print(df_bad.count())

In [ ]:
df_bad.show(5)

In [ ]:
df['date_loan'] = pd.to_datetime(df.date_loan, format='%Y-%m-%d')
df['date_acc'] = pd.to_datetime(df.date_acc, format='%Y-%m-%d')
df['days_between'] = df['date_loan'] - df['date_acc']
df_good = df.loc[(df['status'] == 'A') | (df['status'] == 'C')]
df_bad = df.loc[(df['status'] == 'B') | (df['status'] == 'D')]

## Análise Gráfica

In [ ]:
df_good_pd = df_good.toPandas()
df_bad_pd = df_bad.toPandas()

In [ ]:
# Amount
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
df_good_pd.amount.hist(bins=20, ax=ax1, label='good', color='green', alpha=0.6)
df_bad_pd.amount.hist(bins=20, ax=ax2, label='bad', color='red', alpha=0.6)
ax1.set_title('Loan Amount')
ax2.set_title('Loan Amount')
ax1.legend()
ax2.legend()
plt.show()

In [ ]:
# duration
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
df_good_pd.duration.hist(bins=20, ax=ax1, label='good', color='green', alpha=0.6)
df_bad_pd.duration.hist(bins=20, ax=ax2, label='bad', color='red', alpha=0.6)
ax1.legend()
ax2.legend()
plt.show()

In [ ]:
# payments
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
df_good_pd.payments.hist(bins=20, ax=ax1, label='good', color='green', alpha=0.6)
df_bad_pd.payments.hist(bins=20, ax=ax2, label='bad', color='red', alpha=0.6)
ax1.legend()
ax2.legend()
plt.show()

In [ ]:
# frequency
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
df_good_pd[['account_id', 'frequency']].groupby('frequency').count().plot(kind='bar', color='green', rot=30, ax=ax1, alpha=0.6, legend=False)
df_bad_pd[['account_id', 'frequency']].groupby('frequency').count().plot(kind='bar', color='red', rot=30, ax=ax2, alpha=0.6, legend=False)
ax1.legend()
ax2.legend()
plt.show()

In [ ]:
from pyspark.sql.functions import col, regexp_extract


titanic_df = titanic_df.withColumn("Initial", regexp_extract(col("Name"),"([A-Za-z]+)\.",1))

titanic_df.show()

In [ ]:
titanic_df.select("Initial").distinct().show()


In [ ]:
titanic_df = titanic_df.replace(['Mlle','Mme', 'Ms', 'Dr','Major','Lady','Countess','Jonkheer','Col','Rev','Capt','Sir','Don'],
               ['Miss','Miss','Miss','Mr','Mr',  'Mrs',  'Mrs',  'Other',  'Other','Other','Mr','Mr','Mr'])

In [ ]:
titanic_df.select("Initial").distinct().show()

In [ ]:
titanic_df.groupby('Initial').avg('Age').collect()


In [ ]:
from pyspark.sql.functions import col

titanic_df.filter(col("Initial") == 'Miss').select("Name", "Age", "Initial").show(5)

titanic_df.filter(titanic_df.Initial == 'Miss').select("Name", "Age", "Initial").show(5)

In [ ]:
from pyspark.sql.functions import when

titanic_df = titanic_df.withColumn("Age",when((titanic_df["Initial"] == "Miss") & (titanic_df["Age"].isNull()), 22).otherwise(titanic_df["Age"]))
titanic_df = titanic_df.withColumn("Age",when((titanic_df["Initial"] == "Other") & (titanic_df["Age"].isNull()), 46).otherwise(titanic_df["Age"]))
titanic_df = titanic_df.withColumn("Age",when((titanic_df["Initial"] == "Master") & (titanic_df["Age"].isNull()), 5).otherwise(titanic_df["Age"]))
titanic_df = titanic_df.withColumn("Age",when((titanic_df["Initial"] == "Mr") & (titanic_df["Age"].isNull()), 33).otherwise(titanic_df["Age"]))
titanic_df = titanic_df.withColumn("Age",when((titanic_df["Initial"] == "Mrs") & (titanic_df["Age"].isNull()), 36).otherwise(titanic_df["Age"]))


In [ ]:
titanic_df.groupBy("Embarked").count().show()


In [ ]:
titanic_df = titanic_df.na.fill({"Embarked" : 'S'})

In [ ]:
titanic_df.groupBy("Embarked").count().show()

In [ ]:
titanic_df.describe("Cabin").show()

In [ ]:
titanic_df = titanic_df.drop("Cabin")

titanic_df.printSchema()


In [ ]:
titanic_df = titanic_df.withColumn("Family_Size",col('SibSp')+col('Parch'))

In [ ]:
from pyspark.sql.functions import col


titanic_df.groupBy("Family_Size").count().show()


In [ ]:
from pyspark.sql.functions import asc

titanic_df.select("Name", "Family_Size").orderBy(col("Family_Size").desc()).show()


In [ ]:
from pyspark.sql.functions import lit

titanic_df = titanic_df.withColumn('Alone',lit(0))


In [ ]:
from pyspark.sql.functions import lit


titanic_df = titanic_df.withColumn('Alone',lit(0))
titanic_df = titanic_df.withColumn("Alone",when(titanic_df["Family_Size"] == 0, 1).otherwise(titanic_df["Alone"]))



In [ ]:
titanic_df.columns


In [ ]:
spark.conf.set("spark.sql.codegen.wholeStage", False)

titanic_df.filter(titanic_df.Age > 70).show()


In [ ]:
from pyspark.sql.functions import desc


titanic_df.orderBy(desc("age")).show(1)


In [ ]:
from pyspark.sql.functions import asc

titanic_df.orderBy(asc("age")).show(1)

In [ ]:
titanic_df.stat.corr("age", "fare")


In [ ]:
titanic_df.stat.corr("age", "family_size")


In [ ]:
# https://spark.apache.org/docs/latest/api/python//reference/pyspark.sql/api/pyspark.sql.DataFrameStatFunctions.html#pyspark.sql.DataFrameStatFunctions
titanic_df.stat.crosstab("Embarked", "PClass").show()

In [ ]:
titanic_df.groupBy('pclass').agg({'fare': 'avg'}).show()

In [ ]:
# UDFs ajudam plugar funções complexas

from pyspark.sql.types import StringType
from pyspark.sql.functions import udf


def uppercase(str):
    return str.upper()

upperCaseUDF = udf(lambda z:uppercase(z),StringType())   

titanic_df.select(upperCaseUDF("Name")).show()